In [14]:
# coding=utf-8
import threading, queue, time, urllib
import requests,re
import pandas as pd

In [2]:
#爬取所有股吧的网址

In [3]:
    #爬取所有沪深的代码
html_hstockcode = "http://guba.eastmoney.com/remenba.aspx?type=1&tab=1" #沪市代码网址
res_html_hstockcode = requests.get(html_hstockcode)
html_sstockcode = "http://guba.eastmoney.com/remenba.aspx?type=1&tab=2" #深市代码网址
res_html_sstockcode = requests.get(html_sstockcode)
res_text_hstockcode = res_html_hstockcode.text
res_text_sstockcode = res_html_sstockcode.text
res_text_allstockcode = res_text_hstockcode + res_text_sstockcode
    #正则表达式提取代码
pattern_hscode = re.compile(r'list,\d*?.html')   
result_hscode = pattern_hscode.findall(res_text_allstockcode)
result_hscode = list(map(lambda x:'http://guba.eastmoney.com/'+x, result_hscode))
result_hscode = list(set(result_hscode))

In [4]:
url_stock = "http://guba.eastmoney.com/list,688981.html"

In [25]:
baseUrl = 'http://guba.eastmoney.com/list,300722_'
url_stockhtml = []
urlQueue = queue.Queue()
for i in range(1, 300):
    url = baseUrl + str(i) + '.html'
    urlQueue.put(url)
 #print(url)
def fetchUrl(urlQueue):
    while True:
        try:
           #不阻塞的读取队列数据
            url = urlQueue.get_nowait()
            i = urlQueue.qsize()
        except Exception as e:
            break
        print ('Current Thread Name %s, Url: %s ' % (threading.currentThread().name, url))
        try:
            response = requests.get(url)
            url_stockhtml.append(response.text)
            responseCode = response.getcode()
        except Exception as e:
            continue
    #if responseCode == 200:
   #抓取内容的数据处理可以放到这里
   #为了突出效果， 设置延时
        #time.sleep(1)
if __name__ == '__main__':
    startTime = time.time()
    threads = []
 # 可以调节线程数， 进而控制抓取速度
    threadNum = 30
    for i in range(0, threadNum):
        t = threading.Thread(target=fetchUrl, args=(urlQueue,))
        threads.append(t)
    for t in threads:
        t.start()
    #for t in threads:
  #多线程多join的情况下，依次执行各线程的join方法, 这样可以确保主线程最后退出， 且各个线程间没有阻塞
        #t.join()
    endTime = time.time()
    print ('Done, Time cost: %s ' % (endTime - startTime))

Current Thread Name Thread-37, Url: http://guba.eastmoney.com/list,300722_1.html Current Thread Name Thread-38, Url: http://guba.eastmoney.com/list,300722_2.html 
Current Thread Name Thread-39, Url: http://guba.eastmoney.com/list,300722_3.html 
Current Thread Name Thread-40, Url: http://guba.eastmoney.com/list,300722_4.html 

Current Thread Name Thread-41, Url: http://guba.eastmoney.com/list,300722_5.html Current Thread Name Thread-42, Url: http://guba.eastmoney.com/list,300722_6.html 

Current Thread Name Thread-43, Url: http://guba.eastmoney.com/list,300722_7.html 
Current Thread Name Thread-44, Url: http://guba.eastmoney.com/list,300722_8.html 
Current Thread Name Thread-45, Url: http://guba.eastmoney.com/list,300722_9.html 
Current Thread Name Thread-46, Url: http://guba.eastmoney.com/list,300722_10.html 
Current Thread Name Thread-47, Url: http://guba.eastmoney.com/list,300722_11.html 
Current Thread Name Thread-48, Url: http://guba.eastmoney.com/list,300722_12.html 
Current Threa

Current Thread Name Thread-61, Url: http://guba.eastmoney.com/list,300722_110.html Current Thread Name Thread-37, Url: http://guba.eastmoney.com/list,300722_111.html Current Thread Name Thread-38, Url: http://guba.eastmoney.com/list,300722_112.html Current Thread Name Thread-41, Url: http://guba.eastmoney.com/list,300722_113.html Current Thread Name Thread-40, Url: http://guba.eastmoney.com/list,300722_114.html 




Current Thread Name Thread-48, Url: http://guba.eastmoney.com/list,300722_115.html 
Current Thread Name Thread-57, Url: http://guba.eastmoney.com/list,300722_116.html Current Thread Name Thread-55, Url: http://guba.eastmoney.com/list,300722_117.html 

Current Thread Name Thread-54, Url: http://guba.eastmoney.com/list,300722_118.html 
Current Thread Name Thread-53, Url: http://guba.eastmoney.com/list,300722_119.html 
Current Thread Name Thread-63, Url: http://guba.eastmoney.com/list,300722_120.html 
Current Thread Name Thread-58, Url: http://guba.eastmoney.com/list,300722_12

In [27]:
len(url_stockhtml)

299

In [28]:
#选择评论区块
def html_ex(html):
    content_list = []
    pattern_1 = re.compile(r'articleh normal_post(.*?)</div>',re.DOTALL)   
    result_1 = pattern_1.findall(html)
    for content in result_1:
        info = content_ex(content)
        if info != []:  
            if info == ["none"]:
                pass
            else:
                content_list.append(content_ex(content))
        else:
            break
    return content_list

In [29]:
def content_ex(content):
    #选择相应内容
    try:
        pattern_2 = re.compile(r'a1">(\d*?)</span>',re.DOTALL)   
        read = pattern_2.findall(content)[0] #阅读数
        pattern_3 = re.compile(r'a2">(\d*?)</span>',re.DOTALL)   
        view = pattern_3.findall(content)[0] #评论数
    except:
        return ["none"]
    pattern_4 = re.compile(r'href.*?>(.*?)</a>',re.DOTALL)   
    con = pattern_4.findall(content)[0] #评论内容
    pattern_5 = re.compile(r'a5">(.*?) .*?</span>',re.DOTALL)   
    time = pattern_5.findall(content)[0] #评论日期
    pattern_6 = re.compile(r'a5">.*? (.*?)</span>',re.DOTALL)   
    minu = pattern_6.findall(content)[0] #评论时间
    stock_content = [read,view,con,time,minu]
    return stock_content

In [30]:
#单个股票的评论信息主函数
all_content_list =[]
for html in url_stockhtml:
    all_content_list = all_content_list + html_ex(html)

In [31]:
#单个股票的评论信息主函数——显示
df = pd.DataFrame(all_content_list, columns=['阅读数','评论数', '评论内容', '评论日期',"评论时间"])
df

,阅读数,评论数,评论内容,评论日期,评论时间
0,24412,78,花颜9.28实盘[上涨]持有新余国科，曙光股份，大宏立，,09-29,15:46
1,118,2,19元多的成本，出来一大半,09-29,15:51
2,34,0,到底管不管这种业绩差的恶意炒作拉高？,09-29,15:51
3,1222,5,谁是军工概念创业板龙头一哥，新余国科、江龙船艇还是,09-29,15:50
4,61,0,新余国科就是科研机构,09-29,15:50
...,...,...,...,...,...
23845,1353,12,往里冲弟兄们，可以买进了，抢购抢购新余国科，满仓干,06-12,11:03
23846,1994,0,新余国科6月12日快速反弹,06-12,10:59
23847,2292,4,该跌不跌必有妖，明天涨停,06-12,10:59
23848,2813,4,散户看好越跌。,06-12,10:59


In [32]:
df.to_excel('新余国科评论.xls')